<a href="https://colab.research.google.com/github/kunwarp29/DPrimeLearning/blob/master/Final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp -r /content/drive/MyDrive/Palmprint-Recognition-master '/content'

In [ ]:
!cp -r '/content/drive/MyDrive/IITD Palmprint V1.zip' '/content'

In [ ]:
!unzip -q '/content/IITD Palmprint V1.zip'

In [ ]:
!du -sh '/content/IITD Palmprint V1'

2.0G	/content/IITD Palmprint V1


In [ ]:
# !pip uninstall -y tensorflow-gpu
!pip install tensorflow==1.14

     |████████████████████████████████| 109.3MB 160kB/s 
     |████████████████████████████████| 3.2MB 22.3MB/s 
     |████████████████████████████████| 491kB 48.7MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np
import random

import tensorflow as tf
import os
import matplotlib.image as mp
import re
import pandas as pd
from termcolor import colored
import sys
import cv2

sys.path.insert(0,'/content/Palmprint-Recognition-master/libs')


#from ./vgg_cnn_f import VGG_CNN_F
from vgg_cnn_f import VGG_CNN_F
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

pic_dim = 224
class_dim = 230


def reduce_var(x, axis=None, keepdims=False):
    m = tf.reduce_mean(x, axis=axis, keep_dims=True)
    devs_squared = tf.square(x - m)
    return tf.reduce_mean(devs_squared, axis=axis, keep_dims=keepdims)


def reduce_std(x, axis=None, keepdims=False):
    return tf.sqrt(reduce_var(x, axis=axis, keepdims=keepdims))


def get_class(name):
    Chot = np.array([0.0] * class_dim)
    Chot[int(name)] = 1.0
    return Chot


def get_class_number(name):
    res = re.findall('[0-9]+', name)

    return int(res[0]) - 1

def data_gen(X, batch_size):
    l = len(X)
    ret = []
    k = l//batch_size
    
def data_gen(X, batch_size):
    l = len(X)
    ret = []
    k = l//batch_size
    for i in range(0, k):
        data = []
        label = []
        for j in range(i*batch_size, (i+1)*batch_size):
            img = cv2.imread(X[j][0], 1)
            img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_NEAREST)
            data.append(img.astype('float64'))
            label.append(X[j][1])
        yield (data, label)

batch_size = 20

image = tf.placeholder(tf.float32, [None, 224, 224, 3])
image = tf.div(tf.subtract(image, tf.reduce_mean(image)), reduce_std(image))

labels = tf.placeholder(tf.float32, [None, 230])
net = VGG_CNN_F({'data': image})

fc7 = net.layers['fc7']

print (fc7.shape)
with tf.variable_scope("kunwar", reuse=tf.AUTO_REUSE):

 w1 = tf.Variable(tf.random_normal(shape=[4096, 100], mean=0, stddev=2 / np.sqrt(4096)), name='feature_layer')
 b1 = tf.Variable(tf.zeros([100]), name='feature_layer_bias')
 output = tf.matmul(fc7, w1) + b1
 w = tf.Variable(tf.random_normal(shape=[100, 230], mean=0, stddev=2 / np.sqrt(100)), name='last_W')
 b = tf.Variable(tf.zeros([230]), name='last_b')
 output1 = tf.matmul(output, w) + b

 prediction = tf.nn.softmax(output1) #eval

 cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output1))
 tf.summary.scalar('Cost', cost)
 opt = tf.train.AdamOptimizer(learning_rate=0.00004)
 train_op = opt.minimize(cost)
 #evaluate model
 correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
 accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


merged = tf.summary.merge_all()
saver = tf.train.Saver()
init = tf.initialize_all_variables()

epoch = 20
save_path = './log/weights/cnn_f_IITD_fine_tune.ckpt'
log_path = './log/summaries/IITD/'
extract = True
step = 0
loadw = False
train = True


with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter(log_path, sess.graph)
    '''if loadw:
        saver.restore(sess, save_path)

        print ('Model was restored.')
        im_path = './data/new_right/'
        if extract:
            images = [
                [mp.imread('./data/new_right/' + name), get_class_number(name), name]
                for name in os.listdir('./data/new_right/')
            ]
            features = []
            for i in range(len(images)):
                f = sess.run(output, feed_dict={image: [images[i][0]]})
                features.append(list(f[0]) + [images[i][1]] + [images[i][2]])


            features = pd.DataFrame(features, columns=['vgg_' + str(i) for i in range(100)] + ['vgg_label'] + ['name'])
            features.to_csv('CNNF_features_IITD_train.csv', index=False)

    else:
        sess.run(init)
        net.load('./vgg_cnn_f.npy', sess)
        print ("Model's weights were loaded.")'''
    sess.run(tf.global_variables_initializer())
    if train:
        image_paths = [
            ['/content/IITD Palmprint V1/Left Hand/' + name, get_class(get_class_number(name))]
            for name in os.listdir('/content/IITD Palmprint V1/Left Hand/')
        ]
        train_data, test_data = image_paths[:(len(image_paths)*70)//100], image_paths[(len(image_paths)*30)//100 :]
        %%shell
        for i in range(epoch):
            random.shuffle(train_data)
            j = 0
            for xb, yb in data_gen(train_data, batch_size):
                _, c = sess.run([train_op, cost], feed_dict={image: xb, labels: yb})
                # print(np.array(yb).shape, xb[0].shape)
                # if step % 5 == 0:
                summary = sess.run(merged, feed_dict={image: xb, labels: yb})
                #summary_writer.add_summary(summary, step)
               # print (colored('Summaries were updated.', 'magenta'))
                acc = sess.run(accuracy, feed_dict={image: xb, labels: yb})
                if step % 100 == 0:
                    saver.save(sess, save_path)
                    print (colored('Model was saved.', 'magenta'))
                step += 1

                # acc = sess.run(accuracy, feed_dict={image: xb, labels: yb})
                print (colored('Epoch:', 'magenta'), i + 1, '|', colored('Batch:', 'blue'), j + 1, '|', colored('Cost:',
                                                                                                             'green'), c, '|', 'acc ', acc*100)
                print ('_____________________________')
                j += 1
        saver.save(sess, save_path)
        print (colored('Model was saved.', 'magenta'))

        for x_test, y_test in data_gen(test_data, len(test_data)):
            acc = acc = sess.run(accuracy, feed_dict={image: x_test, labels: y_test})
            print('model accuracy : ' , acc*100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

(?, 6, 6, 256)

(?, 4096)
(?, 4096)

(?, 4096)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Model was saved.
Epoch: 1 | Batch: 1 | Cost: 22.230162 | acc  0.0
_____________________________
Epoch: 1 | Batch: 2 | Cost: 33.222015 | acc  0.0
_____________________________
Epoch: 1 | Batch: 3 | Cost: 28.582806 | acc  0.0
_____________________________
Epoch: 1 | Batch: 4 | Cost: 29.228031 | acc  5.000000074505806
_____________________________
Epoch: 1 | Batch: 5 | Cost: 26.273296 | acc  0.0
_____________________________
Epoch: 1 | Batch: 6 | Cost: 28.295603 | acc  0.0
____________